# 第9章: RNN, CNN
（89）は、Transformersを使う

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [46]:
path = '/content/drive/MyDrive/nlp100/'


In [47]:
!unzip /content/drive/MyDrive/nlp100/NewsAggregatorDataset.zip

Archive:  /content/drive/MyDrive/nlp100/NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2pageSessions.csv       
replace __MACOSX/._2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._2pageSessions.csv  
replace newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: newsCorpora.csv         y
y

replace __MACOSX/._newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: __MACOSX/._newsCorpora.csv  
replace readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: readme.txt              
replace __MACOSX/._readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._readme.txt   


In [48]:
# import csv
import pandas as pd

data = '/content/newsCorpora.csv'

publisheres = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
names = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
csv_input = pd.read_csv(data, sep="\t", header=None, names=names)

# https://qiita.com/motoki1990/items/0274d8bcf1a97fe4a869

In [49]:
csv_input["PUBLISHER"].isin(publisheres)
# https://toeming.hatenablog.com/entry/2021/01/31/pandas_list_isin

0         False
1         False
2         False
3         False
4         False
          ...  
422414    False
422415    False
422416    False
422417    False
422418    False
Name: PUBLISHER, Length: 422419, dtype: bool

In [50]:
from sklearn.model_selection import train_test_split

df = csv_input[csv_input["PUBLISHER"].isin(publisheres)]
df = df.sample(frac=1)

# 訓練：検証：評価=8:1:1に分割
x_train, x_valtest = train_test_split(df.values, test_size=0.2, random_state=0)
x_valid, x_test = train_test_split(x_valtest, test_size=0.5, random_state=0)

print("train:", len(x_train))
print("valid:", len(x_valid))
print("test :", len(x_test))

train: 10672
valid: 1334
test : 1334


In [51]:
# カテゴリ名と記事見出しをタブ区切りでファイルに書き込み
# FORMAT: ID \t TITLE \t URL \t PUBLISHER \t CATEGORY \t STORY \t HOSTNAME \t TIMESTAMP

def write_file(fname, x_data):
    with open(fname, "w")as fout:
        for fact in x_data:
            # fact[1] : 見出し
            # fact[4] : カテゴリ
            fout.write(fact[4] + '\t' + fact[1] + '\n')

write_file("train.txt", x_train)
write_file("valid.txt", x_valid)
write_file("test.txt", x_test)

In [52]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [127]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk

nltk.download('punkt')
data_path = '/content/newsCorpora.csv'

publisheres = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
names = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
csv_input = pd.read_csv(data_path, sep="\t", header=None, names=names)

df = csv_input[csv_input["PUBLISHER"].isin(publisheres)]

df = df.sample(frac=1)

# 訓練：検証：評価=8:1:1に分割
x_train, x_valtest = train_test_split(df.values, test_size=0.2, random_state=0)
x_valid, x_test = train_test_split(x_valtest, test_size=0.5, random_state=0)
# option:strtify  ラベルの偏りをなくす

# FORMAT: ID \t TITLE \t URL \t PUBLISHER \t CATEGORY \t STORY \t HOSTNAME \t TIMESTAMP
def write_file(fname, x_data):
    with open(fname, "w")as fout:
        for fact in x_data:
            # fact[1] : 見出し
            # fact[4] : カテゴリ
            # b = business, t = science and technology, e = entertainment, m = health
            if fact[4] == 'b':
                label = '0'
            elif fact[4] == 't':
                label = '1'
            elif fact[4] == 'e':
                label = '2'
            elif fact[4] == 'm':
                label = '3'
            text = " ".join(nltk.word_tokenize(fact[1]))
            fout.write(label + '\t' + text + '\n')

write_file("train.txt", x_train)
write_file("valid.txt", x_valid)
write_file("test.txt", x_test)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import nltk

# 記号を" "に置換、数字を0に統一などの前処理をしてもいい

def write_feature(infname, outfname):
    with open(infname, "r") as fin, open(outfname, "w") as fout1, open(outfname.replace("feature.txt", "label.txt"), "w") as fout2:
        for line in fin:
            try:
                category, title = line.split('\t')
                fout1.write(" ".join(nltk.word_tokenize(title))+'\n')
                fout2.write(category+'\n')
            except:
                continue

write_feature("train.txt", "train.feature.txt")
write_feature("valid.txt", "valid.feature.txt")
write_feature("test.txt", "test.feature.txt")


with open("train.feature.txt", "r") as f1, open("valid.feature.txt", "r") as f2, open("test.feature.txt", "r") as f3:
    x_train = f1.read().split("\n")[:-1]
    x_valid = f2.read().split("\n")[:-1]
    x_test = f3.read().split("\n")[:-1]

with open("train.label.txt", "r") as f1, open("valid.label.txt", "r") as f2, open("test.label.txt", "r") as f3:
    y_train = [int(i) for i in f1.read().split("\n")[:-1]]
    y_valid = [int(i) for i in f2.read().split("\n")[:-1]]
    y_test = [int(i) for i in f3.read().split("\n")[:-1]]

In [142]:
from collections import Counter
freq_counter = Counter()
# data_list = [x_train, x_valid, x_test]
for line in x_train:
    line = line.split(' ')
    freq_counter += Counter(line)

In [143]:
freq_counter

Counter({'EBay': 6,
         'Faces': 13,
         'European': 66,
         'Privacy': 11,
         'Probes': 2,
         'Over': 120,
         'Cyber-Attack': 1,
         'Comcast': 20,
         'Adds': 15,
         'TV': 69,
         'Subscribers': 7,
         'Again': 27,
         ',': 2164,
         'Defying': 1,
         'Industry': 15,
         'Trend': 4,
         'Could': 52,
         'Meg': 6,
         'Ryan': 37,
         'Re-Launch': 1,
         'Career': 8,
         'With': 401,
         "'How": 16,
         'I': 101,
         'Met': 54,
         'Your': 74,
         'Mother': 47,
         "'": 1685,
         'Spin-off': 3,
         '?': 321,
         'Jem': 3,
         'and': 610,
         'the': 625,
         'Holograms': 5,
         ':': 1077,
         'The': 849,
         'Cast': 29,
         'Announced': 1,
         'What': 78,
         'Do': 63,
         'We': 55,
         'Think': 16,
         'So': 38,
         'Far': 9,
         'GLOBAL': 93,
         'ECONOMY-Chin

In [144]:
soted_freq_counter = freq_counter.most_common()

In [145]:
soted_freq_counter

[(',', 2164),
 ('to', 2121),
 ('...', 2084),
 ("'s", 1904),
 ("'", 1685),
 ('in', 1394),
 (':', 1077),
 ('on', 1063),
 ('as', 1034),
 ('UPDATE', 1014),
 ('-', 990),
 ('of', 972),
 ('for', 966),
 ('The', 849),
 ('US', 813),
 ('To', 725),
 ('the', 625),
 ('and', 610),
 ('$', 498),
 ('In', 482),
 ('Of', 470),
 ('at', 450),
 ('a', 428),
 ('(', 420),
 (')', 411),
 ('``', 410),
 ('With', 401),
 ('Is', 396),
 ('A', 392),
 ('For', 382),
 ('And', 363),
 ('with', 355),
 ('after', 349),
 ('?', 321),
 ('New', 321),
 ('Kardashian', 307),
 (';', 284),
 ('On', 261),
 ('Kim', 259),
 ('up', 252),
 ('by', 251),
 ('&', 249),
 ('After', 248),
 ('says', 234),
 ('is', 231),
 ('1', 229),
 ('China', 225),
 ('At', 213),
 ('!', 203),
 ('From', 194),
 ("n't", 188),
 ('Fed', 184),
 ('ECB', 183),
 ('from', 181),
 ('2', 180),
 ('new', 176),
 ('Says', 169),
 ('her', 167),
 ('shares', 158),
 ('Miley', 157),
 ('Cyrus', 157),
 ('data', 148),
 ('West', 145),
 ('It', 140),
 ('.', 139),
 ('May', 139),
 ('Chris', 136),
 ('

In [146]:
# 頻度とidのdictを作成
from collections import defaultdict

freq2id = defaultdict()
for i, word_freq  in enumerate(soted_freq_counter):
    i += 1
    word = word_freq[0]
    freq = word_freq[1]
    if freq < 2:
        freq2id[word] = 0
    else:
        freq2id[word] = i

In [147]:
freq2id["'s"] # id:4のはず

4

In [148]:
freq2id['tracksuit'] # id:0のはず

0

In [188]:
def put_id(line):
    result = []
    for word in line.split(' '):
        if word in freq2id.keys():
            result.append(freq2id[word])
        else:
            result.append(0) # 頻度0の単語として扱う
    return result

x_train_id = []
for line in x_train:
    id_list = put_id(line)
    x_train_id.append(id_list)

# 81. RNNによる予測
ID番号で表現された単語列x=(x1,x2,…,xT)
がある．ただし，T
は単語列の長さ，xt∈ℝV
は単語のID番号のone-hot表記である（V
は単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列x
からカテゴリy
を予測するモデルとして，次式を実装せよ．

**問題文 続きあり**

In [150]:
x_train_id[0]

[2901, 1363, 179, 1631, 6675, 83, 0]

In [181]:
import os
import numpy as np
#乱数固定
def seed_everything(seed: int):
    """seed固定"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)

In [182]:
import torch
from torch import nn
import torch.utils.data as data
# from torchinfo import summary

g = torch.Generator()
g.manual_seed(SEED)

class RNN(nn.Module):
    def __init__(self, vocab_size, emb_size, padding_idx, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
        self.rnn = nn.LSTM(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.rnn(x, h0)
        x = x[:, -1, :]
        logits = self.fc(x)
        return logits

# パラメータの設定
VOCAB_SIZE = len(set(freq2id.values())) + 2  # 辞書のID数 + unknown + パディングID
EMB_SIZE = 300
PADDING_IDX = len(set(freq2id.values())) + 1
OUTPUT_SIZE = 4
HIDDEN_SIZE = 50
NUM_LAYERS = 1

# モデルの定義
model = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
print(model)

RNN(
  (emb): Embedding(9856, 300, padding_idx=9855)
  (rnn): LSTM(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)


In [183]:
x = torch.tensor([x_train_id[0]], dtype=torch.int64)
print(x)
print(x.size())
print(nn.Softmax(dim=-1)(model(x)))

tensor([[2901, 1363,  179, 1631, 6675,   83,    0]])
torch.Size([1, 7])
tensor([[0.2502, 0.2318, 0.2578, 0.2603]], grad_fn=<SoftmaxBackward0>)


  'b': 0，'t':1，'e':2，'m':3

# 82. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [186]:
# ラベルをtensorに変換
train_t = torch.tensor(y_train).long()
valid_t = torch.tensor(y_valid).long()
test_t = torch.tensor(y_test).long()

In [193]:
# # 検証用・評価用をidに変換
# x_valid_id = []
# for line in x_valid:
#     id_list = torch.tensor(put_id(line))
#     x_valid_id.append(id_list)

# x_test_id = []
# for line in x_test:
#     id_list = torch.tensor(put_id(line))
#     x_test_id.append(id_list)

In [216]:
class NewsDataset(data.Dataset):

    def __init__(self, X, y):
        self.X = x
        self.y = y

    def __len__(self):
        """全データサイズを返す"""
        return len(self.y)

    def __getitem__(self, idx):
        """idxに対応するテンソル形式のデータとラベルを取得"""
        print(self.X)
        inputs = torch.tensor(put_id(self.X))
        return inputs, self.y[idx]

In [211]:
train_dataset = NewsDataset(x_train, y_train)
valid_dataset = NewsDataset(x_valid, y_valid)
test_dataset = NewsDataset(x_test, y_test)

In [236]:
#DataLoaderに格納
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def collate_fn(batch): # バッチ内の長さを揃える (揃えないとエラーになる)
    sequences = [x[0] for x in batch]
    labels = torch.LongTensor([x[1] for x in batch])
    x = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=PADDING_IDX)
    return x, labels

# DataLoaderを作成
batch_size = 16

train_loader = data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, generator=g)
valid_loader = data.DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, generator=g)
test_loader = data.DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, generator=g)


In [239]:
# ネットワークのインスタンスを作成
net = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
net.train()

#損失関数の選択
criterion = nn.CrossEntropyLoss()

#最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# モデルをGPUに転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
device

device(type='cpu')

In [240]:
def calc_acc(loader):
    net.eval()
    with torch.no_grad():
        losses = list()
        total = 0
        correct = 0

        for batch in loader:
            x, t = batch

            x = x.to(device)
            t = t.to(device)
            y = net(x)
            pred = torch.argmax(y, dim=-1)

            total += len(x)
            correct += (pred == t).sum().item()
    return correct / total

In [241]:
#学習ループ
max_epoch = 10

for epoch in range(max_epoch):
    train_losses = list()
    # ミニバッチ学習
    for batch in train_loader:

        # バッチサイズ分のサンプルを抽出
        x, t = batch

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)
        loss = criterion(y, t)


        # 誤差逆伝播
        loss.backward()
        optimizer.step()
        train_losses.append(loss)

    # 更新と切り離し、検証データの性能を確認
    with torch.no_grad():
        losses = list()
        for batch in valid_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            loss = criterion(y, t)
            losses.append(loss)
    val_loss = torch.tensor(losses).mean()
    print("Epoch: %02d  val_loss: %.3f" % (epoch+1, val_loss))

Epoch: 01  val_loss: 1.162
Epoch: 02  val_loss: 1.149
Epoch: 03  val_loss: 1.139
Epoch: 04  val_loss: 1.128
Epoch: 05  val_loss: 1.111
Epoch: 06  val_loss: 1.077
Epoch: 07  val_loss: 1.028
Epoch: 08  val_loss: 0.970
Epoch: 09  val_loss: 0.904
Epoch: 10  val_loss: 0.850


# 83. ミニバッチ化・GPU上での学習
問題82のコードを改変し，B
事例ごとに損失・勾配を計算して学習を行えるようにせよ（B
の値は適当に選べ）．また，GPU上で学習を実行せよ．

In [242]:
# ネットワークのインスタンスを作成
net = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
net.train()

#損失関数の選択
criterion = nn.CrossEntropyLoss()

#最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# モデルをGPUに転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
device

device(type='cpu')

In [243]:
#学習ループ
max_epoch = 10

for epoch in range(max_epoch):
    train_losses = list()
    # ミニバッチ学習
    for batch in train_loader:

        # バッチサイズ分のサンプルを抽出
        x, t = batch

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)
        loss = criterion(y, t)


        # 誤差逆伝播
        loss.backward()
        optimizer.step()
        train_losses.append(loss)

    # 更新と切り離し、検証データの性能を確認
    with torch.no_grad():
        losses = list()
        for batch in valid_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            loss = criterion(y, t)
            losses.append(loss)
    val_loss = torch.tensor(losses).mean()
    print("Epoch: %02d  val_loss: %.3f" % (epoch+1, val_loss))

Epoch: 01  val_loss: 1.167
Epoch: 02  val_loss: 1.155
Epoch: 03  val_loss: 1.146
Epoch: 04  val_loss: 1.138
Epoch: 05  val_loss: 1.121
Epoch: 06  val_loss: 1.085
Epoch: 07  val_loss: 1.018
Epoch: 08  val_loss: 0.969
Epoch: 09  val_loss: 0.911
Epoch: 10  val_loss: 0.869


# 84. 単語ベクトルの導入
事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)
を初期化し，学習せよ．

In [245]:
!gzip -d /content/drive/MyDrive/nlp100/GoogleNews-vectors-negative300.bin.gz

In [246]:
import gensim

model_path = '/content/drive/MyDrive/nlp100/GoogleNews-vectors-negative300.bin'
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [252]:
# 学習済み単語ベクトルの取得
# 学習済み単語ベクトルの取得
VOCAB_SIZE = len((freq2id.values())) + 2
EMB_SIZE = 300
weights = np.zeros((VOCAB_SIZE, EMB_SIZE))
words_in_pretrained = 0
for i, word in enumerate(freq2id.keys()):
    try:
        weights[i] = w2v[word]
        words_in_pretrained += 1
    except KeyError:
        weights[i] = np.random.normal(scale=0.1, size=(EMB_SIZE,)) # ランダムに初期化
weights = torch.from_numpy(weights.astype((np.float32)))

print(f'学習済みベクトル利用単語数: {words_in_pretrained} / {VOCAB_SIZE}')
print(weights.size())

学習済みベクトル利用単語数: 15837 / 19700
torch.Size([19700, 300])


In [253]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_size, padding_idx, hidden_size, output_size, num_layers=1, emb_weights=None):
        super().__init__()
        if emb_weights != None:
            self.emb = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx) # ここを更新できるように変更
        else:
            self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
        self.rnn = nn.LSTM(emb_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.rnn(x, h0)
        x = x[:, -1, :]
        logits = self.fc(x)
        return logits

In [254]:
VOCAB_SIZE = len(freq2id.values()) + 2  # 辞書のID数 + unknown + パディングID
EMB_SIZE = 300
PADDING_IDX = len(set(freq2id.values())) + 1
OUTPUT_SIZE = 4
HIDDEN_SIZE = 50
NUM_LAYERS = 1

# ネットワークのインスタンスを作成
net = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
net.train()

#損失関数の選択
criterion = nn.CrossEntropyLoss()

#最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# モデルをGPUに転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
device

device(type='cpu')

In [255]:
#学習ループ
max_epoch = 10

for epoch in range(max_epoch):
    train_losses = list()
    # ミニバッチ学習
    for batch in train_loader:

        # バッチサイズ分のサンプルを抽出
        x, t = batch

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x)
        loss = criterion(y, t)


        # 誤差逆伝播
        loss.backward()
        optimizer.step()
        train_losses.append(loss)

    # 更新と切り離し、検証データの性能を確認
    with torch.no_grad():
        losses = list()
        for batch in valid_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            loss = criterion(y, t)
            losses.append(loss)
    val_loss = torch.tensor(losses).mean()
    print("Epoch: %02d  val_loss: %.3f" % (epoch+1, val_loss))

Epoch: 01  val_loss: 1.165
Epoch: 02  val_loss: 1.152
Epoch: 03  val_loss: 1.141
Epoch: 04  val_loss: 1.131
Epoch: 05  val_loss: 1.112
Epoch: 06  val_loss: 1.072
Epoch: 07  val_loss: 1.002
Epoch: 08  val_loss: 0.949
Epoch: 09  val_loss: 0.905
Epoch: 10  val_loss: 0.860


# 85. 双方向RNN・多層化
順方向と逆方向のRNNの両方を用いて入力テキストをエンコードし，モデルを学習せよ．

h⃖ T+1=0,h⃖ t=RNN←−−−(emb(xt),h⃖ t+1),y=softmax(W(yh)[h→T;h⃖ 1]+b(y))
ただし，h→t∈ℝdh,h⃖ t∈ℝdh
はそれぞれ，順方向および逆方向のRNNで求めた時刻t
の隠れ状態ベクトル，RNN←−−−(x,h)
は入力x
と次時刻の隠れ状態h
から前状態を計算するRNNユニット，W(yh)∈ℝL×2dh
は隠れ状態ベクトルからカテゴリを予測するための行列，b(y)∈ℝL
はバイアス項である．また，[a;b]
はベクトルa
とb
の連結を表す。

さらに，双方向RNNを多層化して実験せよ．

# 86. 畳み込みニューラルネットワーク (CNN)
ID番号で表現された単語列x=(x1,x2,…,xT)
がある．ただし，T
は単語列の長さ，xt∈ℝV
は単語のID番号のone-hot表記である（V
は単語の総数である）．畳み込みニューラルネットワーク（CNN: Convolutional Neural Network）を用い，単語列x
からカテゴリy
を予測するモデルを実装せよ．

**問題文 続きあり**

# 87. 確率的勾配降下法によるCNNの学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題86で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

# 88. パラメータチューニング
問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．

# 89. 事前学習済み言語モデルからの転移学習
事前学習済み言語モデル（例えばBERTなど）を出発点として，ニュース記事見出しをカテゴリに分類するモデルを構築せよ．



In [ ]:
# Transformerを使う
